<a href="https://colab.research.google.com/github/yoshihiroo/programming-workshop/blob/master/QC4U_2022/qc4uchapter2_cirq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QC4U 第2回 Cirq写経翻訳
2022.9.18版

東北大学 [大関先生によるQC4U](https://altema.is.tohoku.ac.jp/QC4U/)の解説コードをもとに、理解を深めるためにCirqへの翻訳をやってみているものです。Cirq初心者ですので、正しくないコードの書き方や理解が間違っているところがあるかも知れませんがご容赦ください。説明の本文は敬意をもってほぼ丸々パクらせていただいております。（掲載については大関先生の了解を得ております。ご承諾ありがとうございます。）

[元にした2022.09.16 第2回の解説コード](https://colab.research.google.com/gist/mohzeki222/3e775df4ee15de10cd5aa6912332abdb/qc4uchapter2.ipynb)

# cirqのインストール

前回に引き続きGoogleが提供するcirqを利用しましょう。
（注意追記）コマンド実行後、WARNINGとともにRESTART RUNTIMEボタンが出てくるのでクリックしてランタイムを再起動する。

In [ ]:
pip install cirq

今回はグローバーの探索アルゴリズムを用いて、振幅を大きくして、所望の量子状態を得るという計算を実行してみます。

In [ ]:
import cirq

今回は5量子ビットを利用して、$2^5$通りの量子状態の中から所望の量子状態を1つ取り出します。
グローバーのアルゴリズムでは、どの量子状態が欲しいのかをリクエストする「オラクル」の存在が必要です。
そのオラクルに、この量子状態が欲しいとチェックしてもらいます。
ただそのチェックが、振幅を大きくするようなものであったらやらせになってしまいますので、
そのオラクルは、あくまで振幅の大きさはそのままに反転だけしてくれるというものとします。

まずその所望の状態に目印をつけるためには、どうしたら良いのかを考えてみましょう。
1つの量子ビットだけだったらちょっと簡単かもしれないので2つ以上を考えましょう。
|00>,|01>,|10>,|11>の4つの状態から、一つを取り出すというわけです。
例えば|11>という量子状態だけを目印をつけることを考えます。
これにはちょうど良い回路があり制御Zゲートというものがあります。



In [ ]:
n=2

qc = cirq.Circuit()
q = cirq.LineQubit.range(n)

まずは2つの量子ビットを用意しました。
これに複数の状態を重ね合わせの状態にするためにアダマール回路を適用します。


In [ ]:
qc.append(cirq.H.on_each(q))

#以下の二行のコマンドと同じ
#qc.append(cirq.H(q[0]))
#qc.append(cirq.H(q[1]))

途中経過を見てみたいときはシミュレータで動かしてみましょう。

In [ ]:
sim = cirq.Simulator()
res = sim.simulate(qc)
print(res)

結果を表示するためには次の通りのコードでよかったですね。

In [ ]:
print(res.final_state_vector.round(5))

それぞれ1/2ですから2乗してみると1/4です。4つの状態のどれも均等な確率になるというわけです。

さてこれに制御Zゲートを適用してみましょう。
czで実装することができます。
どちらが制御量子ビットでどちらが標的量子ビットかを指定して実行します。

In [ ]:
qc.append(cirq.CZ(q[0], q[1]))

途中の回路の状況を見るにはprintでしたね。

In [ ]:
print(qc)

さてこのときに振幅はどうなっているのでしょうか。

In [ ]:
sim = cirq.Simulator()
res = sim.simulate(qc)
print(res)

狙い通り|11>に-1を目印をつけることができました。
これは何をしているのかというと、制御量子ビットが0の時は何もしない。そして制御量子ビットが1の時にはZという回路を作用させるというものです。
|00>と|01>には何もせず、|10>,|11>には何かをするというわけです。
その何かであるZという回路は、0の場合にはそのまま、1のときには-1という目印をつけるというものになっています。
だから|10>は何もせず,|11>は目印をつけるという計算結果になっています。
これが並列的に、|00>,|01>,|10>,|11>とそれぞれを取り出してやらないで**一度だけでできているというところが驚き**です。

### 等価な回路
量子回路にはさまざまなものがありますが、中には同じ計算結果になるものもあります。
その全てを覚えることは難しいかと思いますが、慣れていくと計算結果がわかってくるので、これは別の書き方ができるな、と思うものもあるかもしれません。

特にアダマール回路は、ZとXの役割を反転させることで有名です。
Zは0の場合にそのまま、1の場合には目印をつけてくれました。
アダマール回路は、|0>の場合に|0>＋|1>という重ね合わせの状態を、|1>の場合に|0>-|1>という重ね合わせの状態を作り出してくれます。
そこでXを利用して反転させると|1>-|0>となって、元の状態に比べると係数が-1倍されていることがわかります。これはZの効果と似ていることに気づきます。
しかしこの重ね合わせの状態のままだとややこしいので、元に戻しましょう。
次のような回路を実行してみます。

In [ ]:
qc2 = cirq.Circuit()
q = cirq.LineQubit.range(1)

ここでアダマール回路の後にXを適用し、その後にアダマール回路を施します。


In [ ]:
qc2.append([
    cirq.H(q[0]),
    cirq.X(q[0]),
    cirq.H(q[0])
])

#以下の３行の書き方と同じ
#qc2.append(cirq.H(q[0]))
#qc2.append(cirq.X(q[0]))
#qc2.append(cirq.H(q[0]))

回路の全体像は次の通りです。

In [ ]:
print(qc2)

この計算結果はどのようになるでしょうか。


In [ ]:
res2 = sim.simulate(qc2)
print(res2.final_state_vector.round(5))

何も起きないのは仕方ありませんね。最初の状態が|0>でしたので、そのまま何も起こらなかったのでしょう。
これはZの作用と似ています。
それでは最初の状態を反転した|1>にしてみましょう。

In [ ]:
qc3 = cirq.Circuit()
q = cirq.LineQubit.range(1)
qc3.append(cirq.X(q[0]))

さてこれに対して、アダマールとX、そしてアダマールを実行してみよう。

In [ ]:
qc3.append([
    cirq.H(q[0]),
    cirq.X(q[0]),
    cirq.H(q[0])
])

回路の全体像は次の通り。

In [ ]:
print(qc3)

シミュレーターでどんな結果になるのかを見てみましょう。

In [ ]:
res3 = sim.simulate(qc3)
print(res3.final_state_vector)

Zと同じように0の時は何もせず、1の時は-1と係数が反転するようになりました。

これと同じように制御Z回路は、制御X回路から作ることができます。

そのため|11>だけに-1をつけるということは次のようにしても実行できることになります。

In [ ]:
qc4 = cirq.Circuit()
q = cirq.LineQubit.range(2)

これでまずはアダマール回路を適用して重ね合わせの状態を作ります。

In [ ]:
qc4.append(cirq.H.on_each(q))

そして制御Zゲートの代わりに制御Xゲートを使って実行してみましょう。

In [ ]:
qc4.append([
    cirq.H(q[1]),
    cirq.CX(q[0],q[1]),
    cirq.H(q[1])
])

回路の全体像を見てみるとなかなか複雑な格好をし始めてきました。

In [ ]:
from cirq.contrib.svg import SVGCircuit
SVGCircuit(qc4)

それではこの回路を実行してみましょう。

In [ ]:
res4 = sim.simulate(qc4)
print(res4.final_state_vector.round(5))

望み通り、|11>だけの係数を-1に反転することに成功しました！
このようにアダマール回路は、XとZの効果を逆転させることができます。
また|0>の状態に作用させると|0>+|1>という重ね合わせの状態にするところも、量子回路らしい特徴を持った重要な回路であることをうかがわせます。

### 自由な回路設計

目印をつけたいときに11ではない、
他の状態に目印をつけるにはどうしたら良いでしょう。
例えば01に目印をつけたいとしましょう。
その場合には、01を11にしてCZを適用した後に、何事もなかったように元に戻してあげるということを試してみましょう。

In [ ]:
qc5 = cirq.Circuit()
q = cirq.LineQubit.range(2)

まずは同じように重ね合わせの状態を作り、2つ目の量子ビットにXを適用して、01は11になるようにします。
ここで00も10に反転してしまいますが、この場合は目印はつけられないので、
後で元に戻せば怒られません。

In [ ]:
qc5.append(cirq.H.on_each(q))

In [ ]:
qc5.append([
    cirq.X(q[1]),
    cirq.CZ(q[0],q[1]),
    cirq.X(q[1])
])

回路の全体像は次のようなものになります。

In [ ]:
SVGCircuit(qc5)

さてこの実行結果はいかがでしょうか。

In [ ]:
res5 = sim.simulate(qc5)
print(res5.final_state_vector.round(5))
print(res5)

お望み通り01（２番目）に目印をつけることができました。このようにして制御Zを利用して目印をつけることでオラクルを実現することができます。
２量子ビットの場合の制御Zと同様に、複数量子ビットの場合も基本的には|11...11>の状態に目印をつけることができれば同等のことができることに気づきます。


### 複数の量子ビットに対するオラクル

例えば5個の量子ビットを用意して、その莫大な量子状態を重ね合わせの状態にして、どれが良いか探索する準備を行います。

In [ ]:
n = 5
qc6 = cirq.Circuit()
q = cirq.LineQubit.range(n)

この5個の量子ビットそれぞれにアダマール回路を適用すると0と1の重ね合わせ状態が作られます。
いくつも書くのはしんどいので、for文、プログラムではコンピュータに繰り返しの作業をお願いすることができます。

In [ ]:
for k in range(n):
  qc6.append(cirq.H(q[k]))

またはon_each(q)とだけ打つと全ての量子ビットに同じ作用をさせることができます。

In [ ]:
#qc6.append(cirq.H.on_each(q))

これに対して制御Zゲートと同じ挙動を複数の量子ビットにまたがって行うものを多重制御Zゲートと呼びます。
ただその実装はqiskitでそのままでは存在せず、次のように多重制御Xゲート（mcx）を利用して実行する。

In [ ]:
qc6.append(cirq.Z(q[n-1]).controlled_by(*q[0:n-1])) # Cirqでは多重制御Zゲートを使えるので、以下３行が不要
#qc6.append(cirq.H(q[n-1]))
#qc6.append(cirq.X(q[n-1]).controlled_by(*q[0:n-1]))
#qc6.append(cirq.H(q[n-1]))

In [ ]:
SVGCircuit(qc6)

さてこの回路が、|11111>に対してのみ係数が-1に反転してくれれば良い。
それを確認してみましょう。

量子状態がどのようになっているのかを確認するにはいつものようにシミュレータで実行しましょう。

In [ ]:
res6 = sim.simulate(qc6)
print(res6.final_state_vector.round(5))

ちゃんと期待通り、|11111>にのみ係数がかかっていますね。
ちなみに係数の大きさは、全ての振幅が0.17678程度です。
これは$2^5=32$から、$1/\sqrt{32}\approx 0.17678$という計算結果と整合しています。

以上をまとめると、欲しい状態に印をつけてくれるオラクルが作れそうです。
32個の状態の中から何個目のものを取り出したいか、
数字を指定されると、その２進数表示を出す関数を利用します。

In [ ]:
N = 12
binN = format(N, '05b')[::-1] #最上位ビットの位置がqiskitと逆（qiskitは左だが、cirqは右）のため、反転
                              #Qiskitの場合は01100. Cirqの場合は00110
print(binN)

このうち0だったところの量子ビットを反転するXを適用することで、その欲しい状態を|11111>にしてあげることができます。それで多重制御Zゲートをかければ良いですね。

In [ ]:
n = 5
qc7 = cirq.Circuit()

まずは重ね合わせの状態にしましょう。
全ての量子ビットにアダマール回路をかけます。

In [ ]:
qc7.append(cirq.H.on_each(q))

これに対して、binNで0だったところにxを適用します。
ここで逆から読む必要があることに注意してください。

In [ ]:
for k in range(n):
  if binN[n-k-1] == "0":
    qc7.append(cirq.X(q[k]))

そしてその後に多重制御Zゲートを実行しましょう。

In [ ]:
qc7.append(cirq.Z(q[n-1]).controlled_by(*q[0:n-1]))

そしてXを適用したところにもう一度同じ作用を実行して、元に戻してあげましょう。

In [ ]:
for k in range(n):
  if binN[n-k-1] == "0":
    qc7.append(cirq.X(q[k]))

回路の全体像は次の通りです。

In [ ]:
SVGCircuit(qc7)

In [ ]:
res7 = sim.simulate(qc7)
print(res7.final_state_vector)

長すぎるので、結果が省略されてしまいますね。

12番目の係数が負になっていることを確認すれば良いので、直接みてみましょう。

In [ ]:
print(res7.final_state_vector[12])

無事にオラクルを作ることに成功しました。

ここまでに作った自作のオラクル回路をまとめておいて、後で利用できる形にしておきましょう。
そのためにはpythonで自作関数を作ります。
$2^n-1$を最大値としてさまざまな数値を受け取れるようにif文で例外処理を設けつつ、
以下のように関数を定義します。

In [ ]:
from sympy.core.evalf import quadosc

def oracle(N,n):
  if N > 2**n - 1:
    N = 2**n-1

  qc = cirq.Circuit()
  q = cirq.LineQubit.range(n)

  binN = format(N, '05b')[::-1] #最上位ビットの位置がqiskitと逆（qiskitは左だが、cirqは右）のため、反転
  
  #所望の状態を|111...1>に
  for k in range(n):
    if binN[n-k-1] == "0":
      qc.append(cirq.X(q[k]))

  #多重制御Zゲート
  qc.append(cirq.Z(q[n-1]).controlled_by(*q[0:n-1]))

  #|111...1>から所望の状態に戻す
  for k in range(n):
    if binN[n-k-1] == "0":
      qc.append(cirq.X(q[k]))

#  Uoracle = qc.to_gate()
#  Uoracle.name = "Uoracle"

  return qc

最後に用意した量子回路を、ひとつのまとまった回路にするという、to_gate()を利用します。

早速このまとめられた回路を利用して、プログラムを書くと以下のようにコンパクトにすることができます。

In [ ]:
n = 5
N = 8

qc8 = cirq.Circuit()
q = cirq.LineQubit.range(n)

qc8.append(cirq.H.on_each(q))
oracle_gate = oracle(N,n)
qc8.append(oracle_gate)


qc8.append(oracle_gate,qr)
というところが特徴的ですね。自作回路を、どの量子ビットにかけるかを指定しています。
全てにかけたい場合はqrでOKです！

出来上がった回路を見てみると次のようになります。

In [ ]:
SVGCircuit(qc8)

中身がわからないまとめられた形にされてしまいましたが、自分で作ったものですから不安はありませんね。
と言っても正しく動作しているかどうかはしっかり確かめたいところです。
こうした自作回路の場合はそのままシミュレータを利用することができません。
transpileを利用して、シミュレータで実行するために利用できる回路で書き直してもらいましょう。

In [ ]:
res8 = sim.simulate(qc8)
print(res8.final_state_vector.round(5))

In [ ]:
print(res8.final_state_vector[N].round(5))

正しくNのところだけ振幅を負にすることができました！

### diffuserの作成

さて次はこの振幅が負になった量子状態から、振幅を増幅させるということを考えます。
重ね合わせの状態にある量子状態から、探している量子状態の振幅だけを反転させました。
これは重ね合わせの状態と言っても崩れているものだと言えます。

簡単のため2量子ビットで重ね合わせの状態を作ってみます。

In [ ]:
n = 2
qc9 = cirq.Circuit()
q = cirq.LineQubit.range(n)
qc9.append(cirq.H.on_each(q))

この状況でシミュレーションを実行すると全ての状態が同じ係数で重ね合わせの状態となっています。

In [ ]:
res9 = sim.simulate(qc9)
print(res9.final_state_vector.round(5))

これで重ね合わせの状態を解いてみるとどうでしょうか。
アダマール回路をもう一度全ての量子ビットについて実行します。

In [ ]:
qc9.append(cirq.H.on_each(q))

回路全体は次の通りです。

In [ ]:
print(qc9)

シミュレーションしてみると、次のように変化していることがわかります。

In [ ]:
res9 = sim.simulate(qc9)
print(res9.final_state_vector.round(5))

|00>という状態になります。
一方で重ね合わせの状態を少し崩してみることにしましょう。

In [ ]:
n = 2
qc10 = cirq.Circuit()
q = cirq.LineQubit.range(n)
qc10.append(cirq.H.on_each(q))
qc10.append(cirq.CZ(q[0],q[1]))
qc10.append(cirq.H.on_each(q))

途中で制御Zゲートを導入しているので、|11>だけ振幅が反転したという状況です。

In [ ]:
res10 = sim.simulate(qc10)
print(res10)

すると先ほどとは異なり、|00>だけではなく、他の状態にも確率振幅が出てくるようになりました。
まず|00>が重ね合わせの状態が完璧にできている場合に、アダマール回路を経ると反応すると考えると、重ね合わせの状態から崩れた影響で、|01>,|10>,|11>にも確率振幅が漏れ出したということが言えます。
その漏れ出した確率振幅は、目印をつけるために振幅をマイナスにした影響で生じたはずです。
|00>だけをマイナスにして、|01>,|10>,|11>をそのままにして元に戻すとどうなるでしょうか。
溢れた水を戻すという。しかも重ね合わせの状態の部分を変化させてみるという発想です。

|11>だけに-1をするのは制御Z回路で実行できました。
Xをそれぞれの量子ビットに適応して、|00>を|11>に変化させておいて制御Z回路を実行し、
元に戻せばできるはずです。

In [ ]:
n = 2
qc11 = cirq.Circuit()
q = cirq.LineQubit.range(n)
#重ね合わせを作る
qc11.append(cirq.H.on_each(q))
#11に傷がつく
qc11.append(cirq.CZ(q[0],q[1]))
qc11.append(cirq.H.on_each(q))

#00を11にする
qc11.append(cirq.X.on_each(q))
#11の振幅を反転（元々00）
qc11.append(cirq.CZ(q[0],q[1]))
#11を00にして元に戻すことで00の振幅が反転
qc11.append(cirq.X.on_each(q))


In [ ]:
res11 = sim.simulate(qc11)
print(res11)

それではこの状態をアダマール回路で元に戻してみましょう。
多少は変化しているでしょうか。

In [ ]:
#重ね合わせを解く
qc11.append(cirq.H.on_each(q))

In [ ]:
res11 = sim.simulate(qc11)
print(res11)

お、おおっと。|11>の状態だけが取り出せた...。
まさか。まさか。できたのではないか？
重ね合わせの状態を減らして、漏れ出した係数が、印をつけたことによる影響だから保存しておこうという発想でした。
これを一般的な量子ビット数で適用できるように自作回路にしておきましょう。

In [ ]:
def diffusion(n):
  qc = cirq.Circuit()
  q = cirq.LineQubit.range(n)

  #アダマールを実行して、重ね合わせの状態の崩れ具合を調べられるようにする。
  qc.append(cirq.H.on_each(q))

  #多重制御Zゲートで|0...0>だけの係数を-1にするために|0...0>を|1...1>に
  qc.append(cirq.X.on_each(q))
    
  #多重制御Zゲート
  qc.append(cirq.Z(q[n-1]).controlled_by(*q[0:n-1]))

  #元に戻して|1...1>を|0...0>にする。
  qc.append(cirq.X.on_each(q))

  #アダマールを適用して、重ね合わせの状態を解いてみる
  qc.append(cirq.H.on_each(q))   

#  Udiff = qc.to_gate()
#  Udiff.name = "Udiff"

  return qc

オラクル（Uoracle）と拡散（Udiff）を実行して、どのような結果が出るのかみてみよう。
n=5量子ビット、N=12の数字で指し示された量子状態を探索するというものである。

In [ ]:
n = 5
N = 8

qc12 = cirq.Circuit()
q = cirq.LineQubit.range(n)

#重ね合わせの状態
qc12.append(cirq.H.on_each(q))
oracle_gate = oracle(N,n)
qc12.append(oracle_gate)
diff_gate = diffusion(n)
qc12.append(diff_gate)


こうした自作回路の場合にはそのままstate_vectorのシミュレータは使えないので、
transpileを使って、等価な回路でシミュレータないしはマシンが使える回路に置き換えましょう。

In [ ]:
res12 = sim.simulate(qc12)
print(res12)

In [ ]:
print(res12.final_state_vector[N].round(5))

N番目のものだけ振幅が大きくなっていることがわかります。

このアルゴリズムは繰り返していくと、この確率は次第に大きくなって行きます。
例えば3回繰り返す以下のような量子回路を作ってみましょう。

In [ ]:
n = 5
N = 8
Tall = 3

qc13 = cirq.Circuit()
q = cirq.LineQubit.range(n)
#重ね合わせの状態
qc13.append(cirq.H.on_each(q))
oracle_gate = oracle(N,n)
diff_gate = diffusion(n)
for k in range(Tall):
  qc13.append(oracle_gate)
  qc13.append(diff_gate)


シミュレーションを走らせてみましょう。

In [ ]:
res13 = sim.simulate(qc13)
print(res13)

N番目の確率振幅を見てみましょう。

In [ ]:
print(res13.final_state_vector[N].round(5))

圧倒的な大きさを占めていることがわかります。
ここまで確率振幅が大きければ、測定をした場合にはほとんどNで示される状態が登場することになります。

In [ ]:
qc13.append(cirq.measure(q, key='m'))
SVGCircuit(qc13)

In [ ]:
res13 = sim.run(qc13, repetitions=1000)
counts = res13.histogram(key='m')

In [ ]:
print(counts)

確かにNに相当する状態が登場する確率が非常に高いことがわかります。

In [ ]:
import matplotlib.pyplot as plt
cirq.plot_state_histogram(res13, plt.subplot())
plt.show()